In [2]:
#Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('rainfall.csv')

#Parse date column
df["datetime"] = pd.to_datetime(df["datetime"], format="%d/%m/%y", errors='coerce')
df = df.dropna(subset=["datetime", "solarradiation"]).reset_index(drop=True)

#Create lag features
n_lags = 3
for lag in range(1, n_lags + 1):
    df[f'solarradiation_lag_{lag}'] = df['solarradiation'].shift(lag)

df = df.dropna().reset_index(drop=True)

In [4]:
features = [f'solarradiation_lag_{i}' for i in range(1, n_lags + 1)]
X = df[features]
y = df['solarradiation']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)  

X_train shape: (564, 3)
X_test shape: (141, 3)
y_train shape: (564,)
y_test shape: (141,)
